In [1]:
import pandas as pd
from Investar import Analyzer
from datetime import datetime
import pymysql

class DualMomentum:
    def __init__(self):
        """생성자: KRX 종목코드(codes)를 구현하기 위한 marketDB객체 생성"""
        self.mk = Analyzer.MarketDB()
    
    def get_rltv_momentum(self, start_date, end_date, stock_count):
        #사용자로부터 시작일과 종료일을 입력받아 실제로 DB에서 조회되는 일자로 변경
        connection = pymysql.connect(host='localhost', port=3306, db='investar', user='root', password='qwe0611', autocommit=True)
        cursor = connection.cursor()

        sql = f"SELECT max(date) FROM daily_price where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print("start_date : {} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')

        sql = f"SELECT max(date) FROM daily_price where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print("end_date : {} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')

        #종목별 수익률 계산
        rows=[]
        columns=['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(self.mk.codes):
            sql = f"select close from daily_price where code = '{code}' and date = '{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price where code = '{code}' and date = '{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price/old_price -1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price, returns])

        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company','old_price','new_price','returns']]
        df = df.sort_values(by='returns',ascending = False)
        df = df.head(stock_count)
        df.index = pd.Index(range(stock_count))

        connection.close()
        print(df)
        print(f"\nRelative momentum ({start_date} ~ {end_date}) : {df['returns'].mean():.2f}%\n")
        return df
    
    def get_abs_momentum(self, rltv_momentum, start_date, end_date):
        """특정 기간 동안 상대적 모멘텀에 투자했을 때의 평균 수익률 구함(절대적 모멘텀)"""
        stockList = list(rltv_momentum['code'])
        connection = pymysql.connect(host='localhost', port=3306, db='Investar', user = 'root', password = 'qwe0611', autocommit=True)
        cursor = connection.cursor()
        
        #입력한 날자를 DB 날짜로 변경
        sql = f"SELECT max(date) FROM daily_price where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print("start_date : {} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')

        sql = f"SELECT max(date) FROM daily_price where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print("end_date : {} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')
        
        #상대모멘텀의 종목별 수익률을 구해서 2차원 리스트 형태로 추가
        rows=[]
        columns=['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(stockList):
            sql = f"select close from daily_price where code = '{code}' and date = '{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price where code = '{code}' and date = '{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price/old_price -1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price, returns])
        
        #절대모멘텀 데이터프레임 생성 후 수익률 순으로 출력
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company','old_price','new_price','returns']]
        df = df.sort_values(by='returns',ascending = False)
        connection.close()
        print(df)
        print(f"\nAbsolute momentum ({start_date} ~ {end_date}) : {df['returns'].mean():.2f}%")
        
        return

ModuleNotFoundError: No module named 'Investar'

In [8]:
dm = DualMomentum()
rm = dm.get_rltv_momentum('2019-07-01', '2020-09-06', 40)
am = dm.get_abs_momentum(rm, '2020-08-06', '2020-09-06')

      code     company  old_price  new_price      returns
0   019170        신풍제약       6870     146000  2025.181951
1   096530          씨젠      26350     233600   786.527514
2   950130      엑세스바이오       3830      32800   756.396867
3   285130       SK케미칼      58300     404500   593.825043
4   205470        휴마시스       1860      12400   566.666667
5   101490      에스앤에스텍       7010      43800   524.821683
6   268600        셀리버리      37750     233900   519.602649
7   011000      진원생명과학       4450      26850   503.370787
8   196170        알테오젠      35950     214200   495.827538
9   290650      엘앤씨바이오      22350     132900   494.630872
10  084650       랩지노믹스       6120      36250   492.320261
11  253840         수젠텍       7160      41900   485.195531
12  131290        티에스이       7380      37750   411.517615
13  018000         유니슨       1240       5980   382.258065
14  137400         피엔티       5030      24100   379.125249
15  131370        알서포트       4065      18300   350.184502
16  012790    